In [1]:
# !pip install Nasdaq-Data-Link
# !pip install yfinance

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nasdaqdatalink as ndlink
import yfinance as yf
from datetime import datetime

interval = (
    datetime(2013, 3, 1),
    datetime(2018, 1, 1)
)

index = pd.date_range(start=interval[0].strftime('%Y-%m-%d'), end=interval[1].strftime('%Y-%m-%d'), freq='D')

# OHLC Bitcoin

In [2]:
btc = pd.read_csv('bitcoin.csv')
btc['Date'] = pd.to_datetime(btc['Date'], format='%b %d, %Y')
btc.set_index('Date', inplace=True)
btc.sort_index(inplace=True)
btc.rename(columns={'Price': 'Close'}, inplace=True)
btc = btc[['Close', 'Open', 'High', 'Low']]
btc.head()

,Close,Open,High,Low
Date,,,,
2013-01-01,13.3,13.5,13.6,13.2
2013-01-02,13.3,13.3,13.4,13.2
2013-01-03,13.4,13.3,13.5,13.3
2013-01-04,13.5,13.4,13.5,13.3
2013-01-05,13.4,13.5,13.6,13.3


# Bitcoin Blockchain Data

In [3]:
authtoken = 'xx8e6Zzbphz_MLnS3wM_'

symbols = {
    'BCHAIN/TRFUS': 'Bitcoin Total Transaction Fees USD', #Bitcoin Total Transaction Fees USD
    'BCHAIN/TRVOU': 'Bitcoin USD Exchange Trade Volume', #Bitcoin USD Exchange Trade Volume
    'BCHAIN/HRATE': 'Bitcoin Hash Rate', #Bitcoin Hash Rate
    'BCHAIN/CPTRA': 'Bitcoin Cost Per Transaction', #Bitcoin Cost Per Transaction
    'BCHAIN/NTRAN': 'Bitcoin Number of Transactions', #Bitcoin Number of Transactions
}

data = {}
for symbol in symbols.keys():
    query = ndlink.get(symbol, authtoken=authtoken)
    query = query.loc[interval[0].strftime('%Y-%m-%d'):interval[1].strftime('%Y-%m-%d'),:]
    data[symbols[symbol]] = query

blockchain = pd.concat(list(data.values()), axis=1)
blockchain.columns = list(data.keys())
blockchain = blockchain.reindex(index)
blockchain.head()

,Bitcoin Total Transaction Fees USD,Bitcoin USD Exchange Trade Volume,Bitcoin Hash Rate,Bitcoin Cost Per Transaction,Bitcoin Number of Transactions
2013-03-01,1862.939394,4.230278e+06,34.306270,2.143520,60659.0
2013-03-02,1413.137316,3.877987e+05,33.003501,2.639044,49631.0
2013-03-03,1570.901496,1.487783e+06,37.997451,2.602752,54459.0
2013-03-04,2048.096778,4.294412e+05,37.997451,2.715824,56039.0
2013-03-05,2878.072750,5.075535e+06,35.174784,2.938498,66437.0


# External Data

In [4]:
external_symbols = {
    'CL=F': 'Crude Oil',
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'HO=F': 'Heating Oil',
    'NG=F': 'Natural Gas',
    'KC=F': 'Coffee',
    'PL=F': 'Platinum',
    'PA=F': 'Palladium',
    'HG=F': 'Copper',
    'CC=F': 'Cocoa',
    'SB=F': 'Sugar',
    'ZO=F': 'Oat',
    'NQ=F': 'NASDAQ Future',
    'ES=F': 'S&P500 Future',
    '^GDAXI': 'DAX Index',
    '^KS11': 'KOSPI Index'
}

data = {}

ts = yf.download(
    ' '.join(list(external_symbols.keys())),
    start=interval[0].strftime('%Y-%m-%d'),
    end=interval[1].strftime('%Y-%m-%d'),
    threads=True,
)

ts = ts.reindex(index)
external = ts.loc[:, 'Close'].copy()
external.rename(columns=external_symbols, inplace=True)
external.head()

[*********************100%***********************]  16 of 16 completed


,Cocoa,Crude Oil,S&P500 Future,Gold,Copper,Heating Oil,Coffee,Natural Gas,NASDAQ Future,Palladium,Platinum,Sugar,Silver,Oat,DAX Index,KOSPI Index
2013-03-01,2082.0,90.680000,1516.50,1571.900024,3.4815,2.9301,142.899994,3.456,2748.75,718.650024,1571.500000,17.910000,28.451000,402.5,7708.160156,NaN
2013-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-04,2055.0,90.120003,1525.75,1572.099976,3.4830,2.9191,145.949997,3.529,2760.00,712.500000,1564.199951,18.080000,28.457001,403.5,7691.680176,2013.150024
2013-03-05,2076.0,90.820000,1537.00,1574.599976,3.4965,2.9730,140.500000,3.529,2798.25,732.599976,1583.699951,18.190001,28.563000,397.5,7870.310059,2016.609985


In [5]:
dataset = pd.concat((btc, blockchain, external), axis=1)
dataset = dataset.reindex(index)
dataset.fillna(method='ffill', inplace=True)
dataset.head()

,Close,Open,High,Low,Bitcoin Total Transaction Fees USD,Bitcoin USD Exchange Trade Volume,Bitcoin Hash Rate,Bitcoin Cost Per Transaction,Bitcoin Number of Transactions,Cocoa,...,Coffee,Natural Gas,NASDAQ Future,Palladium,Platinum,Sugar,Silver,Oat,DAX Index,KOSPI Index
2013-03-01,34.5,33.4,34.9,32.9,1862.939394,4.230278e+06,34.306270,2.143520,60659.0,2082.0,...,142.899994,3.456,2748.75,718.650024,1571.500000,17.910000,28.451000,402.5,7708.160156,NaN
2013-03-02,34.3,34.5,34.8,33.2,1413.137316,3.877987e+05,33.003501,2.639044,49631.0,2082.0,...,142.899994,3.456,2748.75,718.650024,1571.500000,17.910000,28.451000,402.5,7708.160156,NaN
2013-03-03,34.5,34.3,34.5,33.8,1570.901496,1.487783e+06,37.997451,2.602752,54459.0,2082.0,...,142.899994,3.456,2748.75,718.650024,1571.500000,17.910000,28.451000,402.5,7708.160156,NaN
2013-03-04,36.2,34.5,36.7,34.2,2048.096778,4.294412e+05,37.997451,2.715824,56039.0,2055.0,...,145.949997,3.529,2760.00,712.500000,1564.199951,18.080000,28.457001,403.5,7691.680176,2013.150024
2013-03-05,40.3,36.2,40.7,36.2,2878.072750,5.075535e+06,35.174784,2.938498,66437.0,2076.0,...,140.500000,3.529,2798.25,732.599976,1583.699951,18.190001,28.563000,397.5,7870.310059,2016.609985


In [6]:
dataset.isnull().sum()

Close                                 0
Open                                  0
High                                  0
Low                                   0
Bitcoin Total Transaction Fees USD    0
Bitcoin USD Exchange Trade Volume     0
Bitcoin Hash Rate                     0
Bitcoin Cost Per Transaction          0
Bitcoin Number of Transactions        0
Cocoa                                 0
Crude Oil                             0
S&P500 Future                         0
Gold                                  0
Copper                                0
Heating Oil                           0
Coffee                                0
Natural Gas                           0
NASDAQ Future                         0
Palladium                             0
Platinum                              0
Sugar                                 0
Silver                                0
Oat                                   0
DAX Index                             0
KOSPI Index                           3


In [7]:
dataset.to_csv('dataset.csv')